# Dynamics

# Parameters

$N$: network size.

$N^*$: number of simultaneous spikes triggering full activation of weak network absent inhibition. Ie if $N^*$ neurons spike at once in a weak network, there should be a 0.5 chance that more than $N^*$ will spike at next time step.

$M$: ensemble size.

$M^*$: number of simultaneous spikes w/in ensemble to trigger full activation of ensemble absent inhibition.

$N_{max}$: maximum number of spikes at one timestep.

$q$: connection probability.

$w_{max}$: intra-ensemble cxn weight.

$w_{min}$: non-intra-ensemble cxn weight.

$\theta$: baseline spike threshold.

$a_{\theta}$: post-spike spike-threshold increase.

$\tau_{\theta}$: threshold-adaptation time constant.

## Parameter relationships

$\cfrac{M^*}{M} = \cfrac{N^*}{N} + \alpha_N$: A maximally connected ensemble should require less relative triggering than the full network to activate. Thus, if the drive to the network increases uniformly, the ensemble should activate before the whole network. $\alpha_N$ determines the separation between ensemble and network activation.

How is $N^*$ related to $q$, $w_{min}$, and $\theta$? If $n(t)$ is the number of spikes in the network at $t$, we want 

$p(n(t+1) \geq n(t)|n(t) = N^*) = 0.5$

$p(n(t+1) \geq n(t)|n(t)) = \sum\limits_{n(t+1) = n(t)}^N p(n(t+1)|n(t))$

For independent connection probabilities:

$p(n(t+1)|n(t)) = {N \choose n(t+1)}p(I \geq \theta | n(t))^{n(t+1)}p(I < \theta | n(t))^{N - n(t+1)}$

ie the product of $n(t+1)$ cells getting suprathreshold input and $N-n(t+1)$ cells getting subthreshold input, summed over all subsets of $n(t+1)$ chosen from $N$ total cells. That is just a binomial distribution, however:

$p(n(t+1)|n(t)) = \textrm{Binomial}(n(t+1)|N, p(I \geq \theta|n(t)))$

so letting $\textrm{BiSvl}$ denote the binomial survival function (total probability mass greater than a given value), we have:

$p(n(t+1) \geq n(t)|n(t)) = \textrm{BiSvl}(n(t)|N, p(I \geq \theta|n(t)))$

The probability of a cell getting suprathreshold input given $n(t)$ spikes on the previous timestep is:

$p(I \geq \theta | n(t)) = p(k \geq \theta/w_{min}|n(t)) = \sum\limits_{k = \textrm{int}(\theta/w_{min})}^{n(t)}p(k|n(t))$

ie the probability it receives more than $\theta/w_{min}$ excitatory inputs.

Since projections to any given cell from the $n(t)$ previously active cells are independent, we have:

$p(k|n(t)) = \textrm{Binomial}(k|n(t), q)$

so

$p(I \geq \theta|n(t)) = \textrm{BiSvl}(\textrm{int}(\theta / w_{min})|n(t), q)$.

Therefore:

$p(n(t+1) \geq n(t)|n(t)) = \textrm{BiSvl}(n(t)|N, \textrm{BiSvl}(\textrm{int}(\theta/w_{min})|n(t), q))$.

Since the binomial distribution median, mode, and mean are equal to within a value of 1, we can equivalently find the $N^*$ such that:

$E[n(t+1)|n(t) = N^*] = N^*$.

Since $p(n(t+1)|n(t))$ follows a binomial distribution with params $N$ and $p(I \geq \theta|n(t))$, we have simply

$E[n(t+1)|n(t)] = Np(I \geq \theta|n(t))$

such that

$E[n(t+1)|n(t)] = N\textrm{BiSvl}(\textrm{int}(\theta / w_{min})|n(t), q)$.

Our criterion for $N^*$ then becomes:

$N^* = N\textrm{BiSvl}(\textrm{int}(\theta / w_{min})|N^*, q)$ 

or

$\cfrac{N^*}{N} = \textrm{BiSvl}(\textrm{int}(\theta / w_{min})|N^*, q)$.

Our requirement that $\cfrac{M^*}{M} > \cfrac{N^*}{N}$ is then equivalent to:

$\textrm{BiSvl}(\textrm{int}(\theta / w_{max})|M^*, q) > \textrm{BiSvl}(\textrm{int}(\theta / w_{min})|N^*, q)$

# Plasticity